# xgboost local interpretation

- [Online Course](https://www.trainindata.com/p/machine-learning-interpretability)

Here I show how to make local interpretations for a regression model, but the code for classification is the same. We just need to change the model class.

In [1]:
import pandas as pd

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor

### Load data

In [2]:
# load the California House price data from Scikit-learn
X, y = fetch_california_housing(return_X_y=True, as_frame=True)
X = X.drop(columns=["Latitude", "Longitude"])

# display top 5 rows
X.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467


### Split data

In [3]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=0)

X_train.shape, X_test.shape

((14448, 6), (6192, 6))

## XGBoost

In [4]:
# fit model

gbm = XGBRegressor(
    importance_type="gain",
    n_estimators=5,
    max_depth=2,
    random_state=3,
)

gbm.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type='gain',
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=2, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=5, n_jobs=None,
             num_parallel_tree=None, random_state=3, ...)

## Peek at the ensemble

In [5]:
# print structure of tree

booster = gbm.get_booster()
booster.dump_model('dump.raw.txt', with_stats = True)

with open('dump.raw.txt') as file:
    print(file.read())

booster[0]:
0:[MedInc<5] yes=1,no=2,missing=2,gain=6050.60205,cover=14448
	1:[MedInc<3.07990003] yes=3,no=4,missing=4,gain=1505.21838,cover=11310
		3:leaf=-0.214339092,cover=5520
		4:leaf=0.00459926156,cover=5790
	2:[MedInc<7.80049992] yes=5,no=6,missing=6,gain=1120.00781,cover=3138
		5:leaf=0.286677152,cover=2599
		6:leaf=0.761799335,cover=539
booster[1]:
0:[MedInc<5.60379982] yes=1,no=2,missing=2,gain=3219.354,cover=14448
	1:[MedInc<3.55110002] yes=3,no=4,missing=4,gain=1047.51404,cover=12391
		3:leaf=-0.130962819,cover=7264
		4:leaf=0.0461317673,cover=5127
	2:[MedInc<6.80749989] yes=5,no=6,missing=6,gain=305.747559,cover=2057
		5:leaf=0.241858423,cover=1125
		6:leaf=0.474598616,cover=932
booster[2]:
0:[MedInc<4.58330011] yes=1,no=2,missing=2,gain=1641.849,cover=14448
	1:[AveOccup<2.29999995] yes=3,no=4,missing=4,gain=888.338501,cover=10466
		3:leaf=0.108552471,cover=2169
		4:leaf=-0.107041381,cover=8297
	2:[AveOccup<2.79061365] yes=5,no=6,missing=6,gain=463.413208,cover=3982
		5:lea

## Feature contribution

xgboost does not provide the values at the internal nodes, just at the leaves, so we would need to calculate them manually :/

We are not going to do that, we are going to use Eli5 instead, who does the job for us.

## Eli5

If you have problems to import eli5 with the latest version of sklearn, you need to go to the eli5 files that are stored in your local computer, and change a few lines of code as shown in [this PR](https://github.com/eli5-org/eli5/commit/840695d869e47b8e6cc05baca428d24881113fb6)

More details also available [here](https://github.com/eli5-org/eli5/issues/39).

In [6]:
import eli5

## Global explanations

In [7]:
# Feature importance (global)

eli5.show_weights(gbm, feature_names=gbm.feature_names_in_)

Weight,Feature
0.6469,MedInc
0.2162,AveOccup
0.0819,AveRooms
0.0549,HouseAge
0,Population
0,AveBedrms


In [8]:
# let's compare eli's result with that of 
# xgboost

pd.Series(gbm.feature_importances_,
          index=gbm.feature_names_in_).sort_values(ascending=False)

MedInc        0.646933
AveOccup      0.216249
AveRooms      0.081869
HouseAge      0.054949
AveBedrms     0.000000
Population    0.000000
dtype: float32

## Local explanations

In [9]:
# Display a few observations from test set

X_test.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup
14740,4.1518,22.0,5.663073,1.075472,1551.0,4.180593
10101,5.7796,32.0,6.107226,0.927739,1296.0,3.020979
20566,4.3487,29.0,5.930712,1.026217,1554.0,2.910112
2670,2.4511,37.0,4.992958,1.316901,390.0,2.746479
15709,5.0049,25.0,4.319261,1.039578,649.0,1.712401


In [10]:
# pick one observation

sample_id = 14740

X_test.loc[sample_id]

MedInc           4.151800
HouseAge        22.000000
AveRooms         5.663073
AveBedrms        1.075472
Population    1551.000000
AveOccup         4.180593
Name: 14740, dtype: float64

In [11]:
# feature contribution for 1 sample

eli5.show_prediction(
    gbm, 
    X_test.loc[sample_id], 
    feature_names=gbm.feature_names_in_,
)

In [12]:
y_train.mean()

2.0682462451550387

In [13]:
# manual prediction

y_train.mean() + 0.004 - 0.106

1.9662462451550387

In [14]:
# prediction of xgboost for that sample

gbm.predict(X_test.loc[sample_id].to_frame().T)

array([1.9657769], dtype=float32)

In [15]:
# feature contribution for another sample

eli5.show_prediction(
    gbm, 
    X_test.loc[10101], 
    feature_names=gbm.feature_names_in_,
)

In [16]:
# manual prediction

y_train.mean() + 0.708 - 0.155

2.621246245155039

In [17]:
# prediction of xgboost for that sample

gbm.predict(X_test.loc[10101].to_frame().T)

array([2.6209118], dtype=float32)

In [18]:
# if we want to obtain a dataframe instead of the 
# html output

eli5.explain_prediction_df(
    gbm, 
    X_test.loc[10101], 
    feature_names=gbm.feature_names_in_,
)

,target,feature,weight,value
0,y,MedInc,0.708015,5.779600
1,y,<BIAS>,-0.000074,1.000000
2,y,AveOccup,-0.155276,3.020979
